In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

In [2]:
def gap(row):
    if abs(row['dualBound']) > 999999999 or abs(row['primalBound']) > 999999999 or \
            row['dualBound'] * row['primalBound'] < 0:
        return 1
    else:
        num = abs(row['primalBound'] - row['dualBound'])
        den = max(abs(row['primalBound']), abs(row['dualBound']))
        return num/den

In [12]:
def get_dfs(test_fldr, base_fldr, cold_fldr):
    dfs, bdfs, cdfs = {}, {}, {}
    for file_name in os.listdir(test_fldr):
        if not file_name.endswith(".csv"):
            continue
        stem = file_name.split('.')[0]
        df = pd.read_csv(os.path.join(test_fldr, file_name))
        bdf = pd.read_csv(os.path.join(base_fldr, file_name))
        cdf = pd.read_csv(os.path.join(cold_fldr, file_name))
        df['heuristicPrimalBound'] = np.minimum(df['heuristicPrimalBound'], bdf['heuristicPrimalBound'])
        bdf['heuristicPrimalBound'] = np.minimum(df['heuristicPrimalBound'], bdf['heuristicPrimalBound'])
        cdf['heuristicPrimalBound'] = np.minimum(df['heuristicPrimalBound'], cdf['heuristicPrimalBound'])
        df['rootGap'] = (df['heuristicPrimalBound'] - df['rootDualBound']).abs() / \
                        np.maximum(df['heuristicPrimalBound'].abs(), df['rootDualBound'].abs())
        df['gap'] = (df['primalBound'] - df['dualBound']).abs() / np.maximum(df['primalBound'].abs(), df['dualBound'].abs())
        bdf['rootGap'] = (bdf['heuristicPrimalBound'] - bdf['rootDualBound']).abs() / \
                          np.maximum(bdf['heuristicPrimalBound'].abs(), bdf['rootDualBound'].abs())
        bdf['gap'] = (bdf['primalBound'] - bdf['dualBound']).abs() / np.maximum(bdf['primalBound'].abs(), bdf['dualBound'].abs())
        cdf['rootGap'] = (cdf['heuristicPrimalBound'] - cdf['rootDualBound']).abs() / \
                          np.maximum(cdf['heuristicPrimalBound'].abs(), cdf['rootDualBound'].abs())
        cdf['gap'] = (cdf['primalBound'] - cdf['dualBound']).abs() / np.maximum(cdf['primalBound'].abs(), cdf['dualBound'].abs())
        pdf = pd.read_csv(os.path.join('nearest_objective_angles', file_name))
        df = pd.concat([df, pdf], axis=1)
        bdf = pd.concat([bdf, pdf], axis=1)
        cdf = pd.concat([cdf, pdf], axis=1)
        df['nearest objective angle'] = df['nearest objective angle']*(1 - (df["vpcGenerator"] == "PRLP"))
        bdf['nearest objective angle'] = df['nearest objective angle']
        cdf['nearest objective angle'] = df['nearest objective angle']
        dfs[stem] = df
        bdfs[stem] = bdf
        cdfs[stem] = cdf
    return dfs, bdfs, cdfs

get_dfs('warm_start', 'benchmark', 'cold_start')

({'obj_series_1_perturbed':         lpBound  rootDualBoundPreVpc  rootDualBound    dualBound  \
  0   8489.767168          8490.388086    8507.792285  8521.980144   
  1   8483.143526          8483.438010    8501.333337  8516.387835   
  2   8454.254540          8454.308874    8468.024204  8483.944292   
  3   8509.061017          8510.255047    8523.434333  8538.141828   
  4   8455.441869          8456.294712    8470.643472  8485.989523   
  5   8498.516229          8499.173615    8509.620812  8531.507072   
  6   8478.038733          8479.354697    8482.401684  8508.556075   
  7   8416.119113          8417.155975    8417.155975  8446.386004   
  8   8516.048898          8517.516117    8517.516117  8540.558463   
  9   8499.811051          8499.948123    8510.851628  8533.919741   
  10  8454.131074          8454.241829    8454.435260  8485.289925   
  11  8466.615628          8467.133821    8467.133821  8491.055764   
  12  8442.538430          8442.573729    8442.573729  8470.3161

In [13]:
vpc_dfs, baseline_dfs, cold_dfs = get_dfs('warm_start', "benchmark", "cold_start")

In [16]:
# for each instance in compare.index, plot baseline_dfs[rootDualBound] on its own plot
matplotlib.rcParams.update({'font.size': 18})
for instance in vpc_dfs:
    running_total = 0
    fig=plt.figure(figsize=(18, 6))
    ax1 = plt.subplot(2, 5, 1)
    ax2 = plt.subplot(2, 5, 6, sharex = ax1)
    ax3 = plt.subplot(2, 5, 2, sharey = ax1)
    ax4 = plt.subplot(2, 5, 7, sharex = ax3, sharey = ax2)
    ax5 = plt.subplot(2, 5, 3, sharey = ax1)
    ax6 = plt.subplot(2, 5, 8, sharex = ax5, sharey = ax2)
    ax7 = plt.subplot(2, 5, 4, sharey = ax1)
    ax8 = plt.subplot(2, 5, 9, sharex = ax7, sharey = ax2)
    ax9 = plt.subplot(2, 5, 5, sharey = ax1)
    ax10 = plt.subplot(2, 5, 10, sharex = ax9, sharey = ax2)

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 0]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 0]

    ax1.scatter(running_total + np.arange(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax1.scatter(running_total + np.arange(len(v)), v['rootGap'], label='VPCs via [1]', c='g', alpha=0.5)
    ax1.axes.get_xaxis().set_visible(False)
    ax1.title.set_text("No Change")
    ax1.set_ylabel('Root Node\nOptimality Gap')

    ax2.scatter(running_total + np.arange(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax2.scatter(running_total + np.arange(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='VPCs via [1]', c='g', alpha=0.5)
    ax2.set_ylabel('Root Node Processing\nTime (Seconds)')
    ax2.set_xticks([x for x in running_total + np.arange(len(b)) if not x%2])
    running_total += len(b)

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 1]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 1]
    c = cold_dfs[instance].loc[cold_dfs[instance]["intended angle"] == 1]

    ax3.scatter(running_total + np.arange(len(c)), c['rootGap'], label='VPCs via [1]', c='g', alpha=0.5)
    ax3.scatter(running_total + np.arange(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax3.scatter(running_total + np.arange(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax3.axes.get_xaxis().set_visible(False)
    ax3.axes.get_yaxis().set_visible(False)
    ax3.title.set_text("1 Degree")

    ax4.scatter(running_total + np.arange(len(c)), c['rootDualBoundTime'] + c['vpcGenerationTime'], label='VPCs via [1]', c='g', alpha=0.5)
    ax4.scatter(running_total + np.arange(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax4.scatter(running_total + np.arange(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax4.axes.get_yaxis().set_visible(False)
    ax4.set_xticks([x for x in running_total + np.arange(len(b)) if not x%2])
    running_total += len(b)

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 3]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 3]
    c = cold_dfs[instance].loc[cold_dfs[instance]["intended angle"] == 3]

    ax5.scatter(running_total + np.arange(len(c)), c['rootGap'], label='VPCs via [1]', c='g', alpha=0.5)
    ax5.scatter(running_total + np.arange(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax5.scatter(running_total + np.arange(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax5.title.set_text("3 Degrees")
    ax5.axes.get_xaxis().set_visible(False)
    ax5.axes.get_yaxis().set_visible(False)

    ax6.scatter(running_total + np.arange(len(c)), c['rootDualBoundTime'] + c['vpcGenerationTime'], label='VPCs via [1]', c='g', alpha=0.5)
    ax6.scatter(running_total + np.arange(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax6.scatter(running_total + np.arange(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax6.axes.get_yaxis().set_visible(False)
    ax6.set_xticks([x for x in running_total + np.arange(len(b)) if not x%2])
    running_total += len(b)

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 5]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 5]
    c = cold_dfs[instance].loc[cold_dfs[instance]["intended angle"] == 5]

    ax7.scatter(running_total + np.arange(len(c)), c['rootGap'], label='VPCs via [1]', c='g', alpha=0.5)
    ax7.scatter(running_total + np.arange(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax7.scatter(running_total + np.arange(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax7.title.set_text("5 Degrees")
    ax7.axes.get_xaxis().set_visible(False)
    ax7.axes.get_yaxis().set_visible(False)

    ax8.scatter(running_total + np.arange(len(c)), c['rootDualBoundTime'] + c['vpcGenerationTime'], label='VPCs via [1]', c='g', alpha=0.5)
    ax8.scatter(running_total + np.arange(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax8.scatter(running_total + np.arange(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax8.axes.get_yaxis().set_visible(False)
    ax8.set_xticks([x for x in running_total + np.arange(len(b)) if not x%2])
    running_total += len(b)

    v = vpc_dfs[instance].loc[vpc_dfs[instance]["intended angle"] == 10]
    b = baseline_dfs[instance].loc[baseline_dfs[instance]["intended angle"] == 10]
    c = cold_dfs[instance].loc[cold_dfs[instance]["intended angle"] == 10]

    ax9.scatter(running_total + np.arange(len(c)), c['rootGap'], label='VPCs via [1]', c='g', alpha=0.5)
    ax9.scatter(running_total + np.arange(len(b)), b['rootGap'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax9.scatter(running_total + np.arange(len(v)), v['rootGap'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax9.title.set_text("10 Degrees")
    ax9.axes.get_xaxis().set_visible(False)
    ax9.axes.get_yaxis().set_visible(False)

    ax10.scatter(running_total + np.arange(len(c)), c['rootDualBoundTime'] + c['vpcGenerationTime'], label='VPCs via [1]', c='g', alpha=0.5)
    ax10.scatter(running_total + np.arange(len(b)), b['rootDualBoundTime'] + b['vpcGenerationTime'], label='no VPCs', c='b', alpha=0.5, marker=',')
    ax10.scatter(running_total + np.arange(len(v)), v['rootDualBoundTime'] + v['vpcGenerationTime'], label='Parameterized VPCs', c='r', alpha=0.5)
    ax10.axes.get_yaxis().set_visible(False)
    ax10.set_xticks([x for x in running_total + np.arange(len(b)) if not x%2])
    
    handles, labels = ax1.get_legend_handles_labels()
    more_handles, more_labels = ax3.get_legend_handles_labels()
    handles += more_handles[2:]
    labels += more_labels[2:]
    lgd = fig.legend(handles, labels, loc='lower center', bbox_to_anchor=(.755, -.165))

    text = fig.text(.29, 0.02, "Instance Number", transform=fig.transFigure)

    # fig.tight_layout()
    # plt.savefig(f'{instance}_root_stats.png')
    fig.savefig(f'{instance}_root_stats.png', bbox_extra_artists=(lgd,text), bbox_inches='tight', dpi=300)
    plt.close()